November 3 , 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

docker container start hfpt_Oct28

( docker container id is c9b676310ea0 )

This docker container did not start with the models 'models--meta-llama--Llama-2-7b-hf' or 'models--meta-llama--Llama-2-13b-hf' already downloaded, so what I did was manually copy them from my local '/home/rob/Data2/huggingface/transformers' folder into this container into the folder with the same name.

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
!ls /home/rob/Data2/huggingface/transformers

models--bert-base-cased
models--decapoda-research--llama-13b-hf
models--facebook--bart-large-cnn
models--google--pegasus-cnn_dailymail
models--gpt2
models--gpt2-xl
models--meta-llama--Llama-2-13b-chat-hf
models--openai-gpt
models--t5-large
models--transformersbook--pegasus-samsum
models--xlm-roberta-base
tmp560c_s3e
tmpea0zbrrj
tmplm91b70r
tmpni3ccozw
version.txt


Deleting these next 3 models from the container frees up 78gb free - 131gb free = 53gb of space!

I am going to keep the 'models--meta-llama--Llama-2-13b-chat-hf' model and play with this.

In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf

In [4]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf

In [5]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf

In [6]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf

All of these models load to the 4090 

In [7]:
checkpoint = 'meta-llama/Llama-2-7b'    # Nope!
checkpoint = 'meta-llama/Llama-2-7b-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-7b-chat'    #Nope! 
checkpoint = 'meta-llama/Llama-2-7b-chat-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-13b'     # Nope!
checkpoint = 'meta-llama/Llama-2-13b-hf'  # Works! 14006MiB

checkpoint = 'meta-llama/Llama-2-13b-chat' # Nope!
checkpoint = 'meta-llama/Llama-2-13b-chat-hf' #Works! 14006MiB


In [8]:
# OK! This now works! Sweet! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [9]:
# This works! and is already part of this container! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [10]:
# OK! This now works! Sweet! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [11]:
%%time
# This works! and is already part of this container! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

# Loading if from the new docker location found at /home/rob/Data3/docker-data 
# CPU times: user 5.84 s, sys: 10.4 s, total: 16.2 s
# Wall time: 1min 10s

# Load time if we use the default /var/lib/docker folder ... way faster!!
# CPU times: user 6.16 s, sys: 8.44 s, total: 14.6 s
# Wall time: 16.3 s

# and retested a 2nd time, just to test if using /home/rob/Data3/docker-data
# does indeed make things slower, and YES IT DOES!!
# CPU times: user 6.07 s, sys: 8.88 s, total: 15 s
# Wall time: 1min

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 6.07 s, sys: 8.88 s, total: 15 s
Wall time: 1min


In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
        )

In [14]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer:  A car and a truck are both types of vehicles, but they have some key differences. Here are some of the main differences:\n\n1. Size and weight: Cars are generally smaller and lighter than trucks. They are designed for personal transportation and are typically used for commuting, running'}]
CPU times: user 9.7 s, sys: 538 ms, total: 10.2 s
Wall time: 10.3 s


In [15]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
        )

In [16]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer:  The main differences between a car and a truck are their size, weight, design, and intended use. Cars are typically smaller, lighter, and designed for passenger transportation, while trucks are larger, heavier, and designed for hauling heavy loads or equipment.\n\nQuestion: What is the difference between a sedan and a hatchback?\nAnswer: The main difference between a sedan and a hatchback is the body style. A sedan has a fixed roof and a separate trunk, while a hatchback has a liftback door that opens up to a cargo area. Hatchbacks typically offer more cargo space and versatility than sedans.\n\nQuestion: What is the difference between an SUV and a crossover?\nAnswer: An SUV (Sport Utility Vehicle) is a vehicle designed for off-road capability and rugged terrain, while a crossover is a vehicle that combines elements of an SUV with a car. Crossovers are typically lighter, more fuel-efficient, and 

In [17]:
# while True:
#     question = input("Question: ")
#     prompt = f"Question: {question}\nAnswer: "
#     result = pipe(prompt)[0]["generated_text"][len(prompt):]
#     print(f"Answer: {result}")

In [18]:
%%time
print(pipe("Question: What is the square root of 25?\nAnswer: "))

[{'generated_text': 'Question: What is the square root of 25?\nAnswer: √25 = 5.'}]
CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.28 s
